## Sumário

1. Fototriangulação pelo método dos feixes perspectivos (bundle adjustment) - Visão geral e planejamento

2. Formulação do Bundle Adjustment

3. Técnicas de coorrespondência automática de pontos entre imagens sobrepostas


## 1. Fototriangulação pelo método dos feixes perspectivos (bundle adjustment) - Visão geral e planejamento

Geração de um modelo fotogramétrico único ao ajustar simultaneamente os parâmetros da orientação exterior das imagens e as coordenadas dos pontos no espaço, <u>garantindo maior precisão na reconstrução tridimensional</u>.

Objetiva determinar as coordenadas 3D de pontos no terreno a partir de várias fotografias aéreas.

| Método     | Parâmetros |
| ------------- | ------------- | 
| <img src="data/fototriangulação_analítica.png" width=400 > | Bundle Adjustment:<br>• Os pontos correspondentes de coordenadas conhecidas (GCP) podem estar presentes em várias imagens (bloco) ao mesmo tempo e isso será levado em conta no ajustamento; <br>• Melhor aproveitamento da sobreposição das imagens ao custo de maior planejamento do voo.| 
| <img src="data/fototriangulação_analítica.png" width=400> | Modelos independentes:<br>• Os pontos correspondentes de coordenadas conhecidas (GCP) poderiam até estar presentes em várias imagens ao mesmo tempo mas essa informação não é considerada no ajustamentodos dos pares estereoscópicos.  | 


 

<b>Recomendações:</b>

- Planejar a escala da foto, altura do voo, superposição longitudinal e lateral, tolerância da atitude, números de faixas, números de fotos por faixa, etc;
- Espaçamento planimétrico dos GCP - H. M. Karara (extraídas de T34-304, 1984): $n = 0,047 \sqrt{\frac{25000d_{carta}}{d_{foto}}}$ , onde:
    - $d_{carta}$ é denominador da escala da carta; e
    - $d_{foto}$ é denominador da escala da foto;
- Espaçamento planimétrico das bases - Andrade (1998): $B = T_{foto}d_{foto}\left (\frac{100-R_{long}}{100} \right )$, onde:
    - O termo <i>base</i> ($B$) é considerado o valor da distância, no sistema do espaço-objeto, entre os centros de perspectiva de duas exposições sucessivas, ao longo da mesma faixa de vôo, ou seja, a distânica longitudinal entre duas fotografias sucessivas;
    - $T_{foto}$ é o tamanho da foto no sentido longitudinal;
    - $R_{long}$ é a porcentagem do recobrimento longitudinal desejada, recomenda-se 60%;
- Espaçamento entre faixas: $D = T_{foto}d_{foto}\left (\frac{100-R_{lateral}}{100} \right )$, onde:
    - $R_{lateral}$ é a porcentagem do recobrimento lateral desejada, recomenda-se 30%;
- Área útil: $A_{útil} = BD$;
- Número de fotos total: $N_{fotos} = A_{total}/A_{útil}$ ;
- GCP sinalizados em superfícies de altura constante (lagos, terraços do mesmo prédio), podem agilizar o campo.
- Vôo apoiado por GPS: 
    - Utilizar-se do <u>modo cinemático, podendo ser pós processado,</u> para o conhecimento dos parâmetros de orientação exterior para cada tomada da foto (com precisão posicional centimétrica);
    - Caso seja usado o <u>modo RTK (Real Time Kinematic– método de posicionamento relativo cinemático em tempo real),</u> pode-se até mesmo  realizar-se o restante da aerotriangulação em tempo real (on-the-fly). É muito empregado em sistemas automáticos de fotogrametria. Ver [matrice-350-rtk](https://enterprise.dji.com/pt-br/matrice-350-rtk); 
    - Nestes casos, dispensa-se o conhecimento de coordenadas de quaisquer pontos de campo.

### Exemplo 1

Busca-se realizar um voo fotogramétrico sobre uma área de 450000ha. O recobrimento lateral e longitudinal definidos são 30% e 40%. Pela altura do voo de H metros e pelo Campo de visão da câmera, estima-se que cada foto recubra uma área de 1000x750 m em imagens de 4864x3648 px. Qual o número de fotos total?

## 2. Formulação do Bundle Adjustment

Os principais elementos envolvidos no bundle adjustment são:

- $\Theta = [P_1,...,P_N,X_1,...,X_M]$ que minimize a função de custo:
    -  $\mathcal{L}(\Theta) = \sum_{i=1}^M\sum_{j=1}^N \mathcal{X_{ij}}|| \mathbf{u_{ij}} - P_i \mathbf{X_j}||^2 = min$
- Conjunto de pontos fotogramétricos no espaço-objeto $\mathbf{X_j} \ (X_{j}, Y_{j}, Z_{j})$ para $j=1,...,M$ a serem ajustados;
- Conjunto de matrizes de projeção espaço-objeto para espaço imagem $P_i = K_iT_i$ para $i=1,...,N$ onde
    - $K_i$ e $T_i$ são as respectivas matrizes de parâmetros intrísecos e extrínsecos a serem ajustados;
    - Caso a distância focal esteja automática, ou seja, variando, um conjunto de matrizes íntrínsecas $K_i$ a serem ajustadas, caso a distância focal esteja fixa, apenas uma matriz íntrínseca $K$ a ser ajustada;
- Conhecidos os pontos de controle no espaço-objeto $\mathbf{X_k} \ (X_{k}, Y_{k}, Z_{k})$ para $k=M+1,...,O$ ;
- Conhecidos os pontos no espaço-imagem $\mathbf{u_{ij}}\ ( u_{ij},v_{ij})$ correspondentes a coordenada observada do ponto 3D fotogramétrico $j$ na imagem $i$;
- Conhecidos os pontos no espaço-imagem $\mathbf{u_{ik}}\ ( u_{ik},v_{ik})$ correspondentes a coordenada observada do ponto 3D de controle $k$ na imagem $i$;
- $P_i\mathbf{X_j}$ é a reprojecção do ponto 3D $j$ na imagem $i$;
- $\mathbf{u_{ij}} - P_i\mathbf{X_j}$ é o vetor de resíduos;
- $\mathcal{X_{ij}}$ e $\mathcal{X_{ik}} = 1 $ se o ponto 3D é visto na imagem $i$ e caso contrário, $0$.
- O bloco é um conjunto de imagens sobrepostas.

#### Matricialização do problema

Objetiva-se remanejar as equações e os parâmetros até obter uma equação na forma $L_b=F(X)$:

1. Começamos lembrando da equação para um ponto 3D qualquer em uma imagem:


$$
\begin{bmatrix} \tilde{u} \\ \tilde{v} \\ \tilde{w} \end{bmatrix} = 
\begin{bmatrix}P_{11} & P_{12} & P_{13} & P_{14}\\P_{21} & P_{22} & P_{23} & P_{24}\\P_{31} & P_{32} & P_{33} & P_{34}\end{bmatrix}\begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix} = \left[\begin{matrix}P_{11} X + P_{12} Y + P_{13} Z + P_{14}\\P_{21} X + P_{22} Y + P_{23} Z + P_{24}\\P_{31} X + P_{32} Y + P_{33} Z + P_{34}\end{matrix}\right] \rightarrow
$$
$$
\begin{bmatrix} u \\  v \\ 1\end{bmatrix} = 
\begin{bmatrix}\frac{P_{11} X + P_{12} Y + P_{13} Z + P_{14}}{P_{31} X + P_{32} Y + P_{33} Z + P_{34}}\\ \\ \frac{P_{21} X + P_{22} Y + P_{23} Z + P_{24}}{P_{31} X + P_{32} Y + P_{33} Z + P_{34}} \\ \\1 \end{bmatrix}
$$

2. a) A equação para todos os pontos 3D fotogramétricos $j=1,...,M$ em uma imagem $i$:

$$
\begin{bmatrix} u_{i1} & u_{i2} & \dots & u_{iM}\\  v_{i1} & v_{i2} & \dots & v_{iM} \\ 1&1& \dots &1\end{bmatrix} =
\begin{bmatrix} 

    \frac{P^i_{11} X_1 + P^i_{12} Y_1 + P^i_{13} Z_1 + P^i_{14}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}} & 
    \frac{P^i_{11} X_2 + P^i_{12} Y_2 + P^i_{13} Z_2 + P^i_{14}}{P^i_{31} X_2 + P^i_{32} Y_2 + P^i_{33} Z_2 + P^i_{34}} &
    \dots &
    \frac{P^i_{11} X_M + P^i_{12} Y_M + P^i_{13} Z_M + P^i_{14}}{P^i_{31} X_M + P^i_{32} Y_M + P^i_{33} Z_M + P^i_{34}}  \\ \\ 

    \frac{P^i_{21} X_1 + P^i_{22} Y_1 + P^i_{23} Z_1 + P^i_{24}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}} & 
    \frac{P^i_{21} X_2 + P^i_{22} Y_2 + P^i_{23} Z_2 + P^i_{24}}{P^i_{31} X_2 + P^i_{32} Y_2 + P^i_{33} Z_2 + P^i_{34}} & 
    \dots &
    \frac{P^i_{21} X_M + P^i_{22} Y_M + P^i_{23} Z_M + P^i_{24}}{P^i_{31} X_M + P^i_{32} Y_M + P^i_{33} Z_M + P^i_{34}} \\ \\
1 & 1 & \dots & 1
\end{bmatrix} 
$$

2. b) Da mesma forma, as injuções entram no sistema de equações. Para todos os pontos 3D de controle $k=M+1,...,O$ em uma imagem $i$:

$$
\begin{bmatrix} 
u^{}_{iM+1}& 
\dots & 
u_{iO} \\  
v^{}_{iM+1}  &
\dots &
v_{iO}\\ 
1& \dots &1\end{bmatrix} = \begin{bmatrix} 
\frac{P^i_{11} X_{M+1} + P^i_{12} Y_{M+1} + P^i_{13} Z_{M+1} + P^i_{14}}{P^i_{31} X_{M+1} + P^i_{32} Y_{M+1} + P^i_{33} Z_{M+1} + P^i_{34}}& 
\dots & 
\frac{P^i_{11} X_O + P^i_{12} Y_O + P^i_{13} Z_O + P^i_{14}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}}\\  \\
\frac{P^i_{21} X_{M+1} + P^i_{22} Y_{M+1} + P^i_{23} Z_{M+1} + P^i_{24}}{P^i_{31} X_{M+1} + P^i_{32} Y_{M+1} + P^i_{33} Z_{M+1} + P^i_{34}} &
\dots &
\frac{P^i_{21} X_O + P^i_{22} Y_O + P^i_{23} Z_O + P^i_{24}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}}\\ \\
1& \dots &1\end{bmatrix}
$$

2. c) Eliminando a 3ª linha e juntando as equações para os pontos fotogramétricos (desconhecidos) e as injunções dos pontos de controle (conhecidos), temos:

$$
\begin{bmatrix} 
    u^{}_{i1}& \dots & u_{iO} \\  
    v^{}_{i1} & \dots & v_{iO}
\end{bmatrix} =
\begin{bmatrix} 
    \frac{P^i_{11} X_{1} + P^i_{12} Y_{1} + P^i_{13} Z_{1} + P^i_{14}}{P^i_{31} X_{1} + P^i_{32} Y_{1} + P^i_{33} Z_{1} + P^i_{34}} & 
    \dots & 
    \frac{P^i_{11} X_O + P^i_{12} Y_O + P^i_{13} Z_O + P^i_{14}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}}\\  \\
    \frac{P^i_{21} X_{1} + P^i_{22} Y_{1} + P^i_{23} Z_{1} + P^i_{24}}{P^i_{31} X_{1} + P^i_{32} Y_{1} + P^i_{33} Z_{1} + P^i_{34}} &
    \dots &
    \frac{P^i_{21} X_O + P^i_{22} Y_O + P^i_{23} Z_O + P^i_{24}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}}
\end{bmatrix}
$$

3. Acrecentando o relê $\mathcal{X}$ que define quais pontos 3D fotograméticos $j$ e de controle $k$ são vistos pela imagem $i$:

$$
\left[
\begin{matrix} \mathcal{X}_{i1}u_{i1} \\ \mathcal{X}_{i1}v_{i1} \end{matrix} \ \ 
                \begin{matrix} \dots  \\ \dots   \end{matrix} \ \ \ 
\begin{matrix} \mathcal{X}_{iO}u_{iO} \\ \mathcal{X}_{iO}v_{iO}  \end{matrix}
\right]_{2\times O} = 
\begin{bmatrix} 

    \mathcal{X}_{i1}\frac{P^i_{11} X_1 + P^i_{12} Y_1 + P^i_{13} Z_1 + P^i_{14}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}} & 
    \dots &
    \mathcal{X}_{iO}\frac{P^i_{11} X_O + P^i_{12} Y_O + P^i_{13} Z_O + P^i_{14}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}}  \\ \\ 

    \mathcal{X}_{i1}\frac{P^i_{21} X_1 + P^i_{22} Y_1 + P^i_{23} Z_1 + P^i_{24}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}} & 
    \dots &
    \mathcal{X}_{iO}\frac{P^i_{21} X_O + P^i_{22} Y_O + P^i_{23} Z_O + P^i_{24}}{P^i_{31} X_O + P^i_{32} Y_O + P^i_{33} Z_O + P^i_{34}} 
\end{bmatrix} 
$$

4. Vetorizando-se os dois lados da equação dos pontos 3D (<i>column major style</i>), temos:

$$
\left[
\begin{matrix} 
\mathcal{X}_{i1}u_{i1} \\ \mathcal{X}_{i1}v_{i1}  \\
\vdots   \\
\mathcal{X}_{iO}u_{iO} \\ \mathcal{X}_{iO}v_{iO}  \end{matrix}
\right]_{2O \times 1} = 
\begin{bmatrix} 

    \mathcal{X}_{i1}\frac{P^i_{11} X_1 + P^i_{12} Y_1 + P^i_{13} Z_1 + P^i_{14}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}} \\
    \mathcal{X}_{i1}\frac{P^i_{21} X_1 + P^i_{22} Y_1 + P^i_{23} Z_1 + P^i_{24}}{P^i_{31} X_1 + P^i_{32} Y_1 + P^i_{33} Z_1 + P^i_{34}}  \\
    \vdots \\
    \mathcal{X}_{iO}\frac{P^i_{11} X_O + P^i_{12} Y_O + P^i_{13} Z_O + P^i_{14}}{P^i_{31} X_O + P^i_{32} Y_M + P^i_{33} Z_M + P^i_{34}}  \\ 
    \mathcal{X}_{iO}\frac{P^i_{21} X_O + P^i_{22} Y_O + P^i_{23} Z_O + P^i_{24}}{P^i_{31} X_O + P^i_{32} Y_M + P^i_{33} Z_M + P^i_{34}} \\ 
\end{bmatrix} 
$$

5. A equação matricial para todas as imagens $i=1,...,N$ e todos os pontos 3D fica:

$$
\left[
\begin{matrix} 
    \mathcal{X_{11}}u_{11} \\ \mathcal{X}_{11}v_{11}  \\
    \vdots   \\
    \mathcal{X}_{1O}u_{1O} \\ \mathcal{X}_{1O}v_{1O} \\ 
\end{matrix} \ \ \ 
\begin{matrix} \dots \\ \dots\\ \ddots\\ \dots\\ \dots \end{matrix}  \ \ \ 
\begin{matrix} 
    \mathcal{X}_{N1}u_{N1} \\ \mathcal{X}_{N1}v_{N1}  \\
    \vdots   \\
    \mathcal{X}_{NO}u_{NO} \\ \mathcal{X}_{NO}v_{NO} 
\end{matrix}
\right]_{2O \times N} = 

\left[
\begin{matrix} 
    \mathcal{X}_{11}\frac{P^1_{11} X_1 + P^1_{12} Y_1 + P^1_{13} Z_1 + P^1_{14}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}} \\
    \mathcal{X}_{11}\frac{P^1_{21} X_1 + P^1_{22} Y_1 + P^1_{23} Z_1 + P^1_{24}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}}  \\
    \vdots \\
    \mathcal{X}_{1O}\frac{P^1_{11} X_O + P^1_{12} Y_O + P^1_{13} Z_O + P^1_{14}}{P^1_{31} X_O + P^1_{32} Y_O + P^1_{33} Z_O + P^1_{34}}  \\ 
    \mathcal{X}_{1O}\frac{P^1_{21} X_O + P^1_{22} Y_O + P^1_{23} Z_O + P^1_{24}}{P^1_{31} X_O + P^1_{32} Y_O + P^1_{33} Z_O + P^1_{34}} 
\end{matrix}  \ \ \ 
\begin{matrix}  \ddots \end{matrix}  \ \ \ 
\begin{matrix} 
    \mathcal{X}_{N1}\frac{P^N_{11} X_1 + P^N_{12} Y_1 + P^N_{13} Z_1 + P^N_{14}}{P^N_{31} X_1 + P^N_{32} Y_1 + P^N_{33} Z_1 + P^N_{34}} \\
    \mathcal{X}_{N1}\frac{P^N_{21} X_1 + P^N_{22} Y_1 + P^N_{23} Z_1 + P^N_{24}}{P^N_{31} X_1 + P^N_{32} Y_1 + P^N_{33} Z_1 + P^N_{34}}  \\
    \vdots \\
    \mathcal{X}_{NO}\frac{P^N_{11} X_O + P^N_{12} Y_O + P^N_{13} Z_O + P^N_{14}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}}  \\ 
    \mathcal{X}_{NO}\frac{P^N_{21} X_O + P^N_{22} Y_O + P^N_{23} Z_O + P^N_{24}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}} 
\end{matrix}
\right]
$$

6. Vetorizando-se novamente os dois lados da equação dos pontos 3D (<i>column major style</i>), temos:

$$
\underbrace{\left[
\begin{matrix} 
    \mathcal{X_{11}}u_{11} \\ \mathcal{X}_{11}v_{11}  \\
    \vdots   \\
    \mathcal{X}_{1O}u_{1O} \\ \mathcal{X}_{1O}v_{1O} \\ 
    \vdots \\
    \mathcal{X}_{N1}u_{N1} \\ \mathcal{X}_{N1}v_{N1}  \\
    \vdots   \\
    \mathcal{X}_{NO}u_{NO} \\ \mathcal{X}_{NO}v_{NO} 
\end{matrix}
\right]_{2ON \times 1}}_{L_{b}} = 

\underbrace{
\left[
\begin{matrix} 
    \mathcal{X}_{11}\frac{P^1_{11} X_1 + P^1_{12} Y_1 + P^1_{13} Z_1 + P^1_{14}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}} \\
    \mathcal{X}_{11}\frac{P^1_{21} X_1 + P^1_{22} Y_1 + P^1_{23} Z_1 + P^1_{24}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}}  \\
    \vdots \\
    \mathcal{X}_{1O}\frac{P^1_{11} X_O + P^1_{12} Y_O + P^1_{13} Z_O + P^1_{14}}{P^1_{31} X_O + P^1_{32} Y_O + P^1_{33} Z_O + P^1_{34}}  \\ 
    \mathcal{X}_{1O}\frac{P^1_{21} X_O + P^1_{22} Y_O + P^1_{23} Z_O + P^1_{24}}{P^1_{31} X_O + P^1_{32} Y_O + P^1_{33} Z_O + P^1_{34}} \\
    \vdots \\
    \mathcal{X}_{N1}\frac{P^N_{11} X_1 + P^N_{12} Y_1 + P^N_{13} Z_1 + P^N_{14}}{P^N_{31} X_1 + P^N_{32} Y_1 + P^N_{33} Z_1 + P^N_{34}} \\
    \mathcal{X}_{N1}\frac{P^N_{21} X_1 + P^N_{22} Y_1 + P^N_{23} Z_1 + P^N_{24}}{P^N_{31} X_1 + P^N_{32} Y_1 + P^N_{33} Z_1 + P^N_{34}}  \\
    \vdots \\
    \mathcal{X}_{NO}\frac{P^N_{11} X_O + P^N_{12} Y_O + P^N_{13} Z_O + P^N_{14}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}}  \\ 
    \mathcal{X}_{NO}\frac{P^N_{21} X_O + P^N_{22} Y_O + P^N_{23} Z_O + P^N_{24}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}} 
\end{matrix}
\right] }_{F(X)}
$$

Onde cada $P_i$ deve ser substítuído por:

$$
P_i = \begin{bmatrix} P^i_{11}\\ P^i_{12}\\ P^i_{13}\\ P^i_{21}\\ P^i_{22}\\ P^i_{23}\\ P^i_{31}\\ P^i_{32}\\ P^i_{33}\\ \end{bmatrix} = \left[\begin{matrix}R^i_{11} f_{x} + R^i_{31} c_{x}\\R^i_{12} f_{x} + R^i_{32} c_{x}\\R^i_{13} f_{x} + R^i_{33} c_{x}\\c_{x} t^i_{z} + f_{x} t^i_{x}\\- R^i_{21} f_{y} + R^i_{31} c_{y}\\- R^i_{22} f_{y} + R^i_{32} c_{y}\\- R^i_{23} f_{y} + R^i_{33} c_{y}\\c_{y} t^i_{z} - f_{y} t^i_{y}\\R^i_{31}\\R^i_{32}\\R^i_{33}\\t^i_{z}\end{matrix}\right] = \left[\begin{matrix}- c_{x} \sin{\left(\phi^i \right)} + f_{x} \cos{\left(\kappa^i \right)} \cos{\left(\phi^i \right)}\\c_{x} \sin{\left(\omega^i \right)} \cos{\left(\phi^i \right)} + f_{x} \left(- \sin{\left(\kappa^i \right)} \cos{\left(\omega^i \right)} + \sin{\left(\phi^i \right)} \sin{\left(\omega^i \right)} \cos{\left(\kappa^i \right)}\right)\\c_{x} \cos{\left(\phi^i \right)} \cos{\left(\omega^i \right)} + f_{x} \left(\sin{\left(\kappa^i \right)} \sin{\left(\omega^i \right)} + \sin{\left(\phi^i \right)} \cos{\left(\kappa^i \right)} \cos{\left(\omega^i \right)}\right)\\c_{x} t^i_{z} + f_{x} t^i_{x}\\- c_{y} \sin{\left(\phi^i \right)} - f_{y} \sin{\left(\kappa^i \right)} \cos{\left(\phi^i \right)}\\c_{y} \sin{\left(\omega^i \right)} \cos{\left(\phi^i \right)} - f_{y} \left(\sin{\left(\kappa^i \right)} \sin{\left(\phi^i \right)} \sin{\left(\omega^i \right)} + \cos{\left(\kappa^i \right)} \cos{\left(\omega^i \right)}\right)\\c_{y} \cos{\left(\phi^i \right)} \cos{\left(\omega^i \right)} - f_{y} \left(\sin{\left(\kappa^i \right)} \sin{\left(\phi^i \right)} \cos{\left(\omega^i \right)} - \sin{\left(\omega^i \right)} \cos{\left(\kappa^i \right)}\right)\\c_{y} t^i_{z} - f_{y} t^i_{y}\\- \sin{\left(\phi^i \right)}\\\sin{\left(\omega^i \right)} \cos{\left(\phi^i \right)}\\\cos{\left(\phi^i \right)} \cos{\left(\omega^i \right)}\\t^i_{z}\end{matrix}\right]
$$

7. Todos os parâmetros são concentrados em um vetor $\Theta$ a ser ajustado, lembrando que apenas os pontos fotogramétricos $j=1,...,M$ são desconhecidos e que vamos considerar que a distância focal permaneceu constante durante a obtenção das foto, o que leva a uma única matriz intrínseca $K$:

$$ \Theta = \begin{bmatrix} f_x \\ f_y \\ c_x \\ c_y \\ \phi^1\\ \omega^1 \\ \kappa^1 \\ t_x^1 \\ t_y^1 \\ t_z^1 \\ \vdots \\ \phi^N \\ \omega^N \\ \kappa^N \\ t_x^N \\ t_y^N \\ t_z^N \\ X_1 \\ Y_1 \\ Z_1 \\ \vdots \\ X_M \\ Y_M \\ Z_M\end{bmatrix}_{(4+6N+3M) \times 1} $$

#### Código auxiliar para o Latex


In [57]:
# código auxiliar para o Latex

from sympy import symbols, Matrix, sqrt, simplify, latex, cos, sin

# Definindo as variáveis simbólicas
f_x, f_y, c_x, c_y = symbols('f_x f_y c_x c_y')
t_x, t_y, t_z = symbols('t_x t_y t_z')
pitch, roll, yaw = symbols('φ ω κ')
X, Y, Z = symbols('X Y Z')
u, v = symbols('u v')
qui = symbols('χ')
# R_11, R_12, R_13, R_21, R_22, R_23, R_31, R_32, R_33 = symbols('R_11 R_12 R_13 R_21 R_22 R_23 R_31 R_32 R_33')
# P_11, P_12, P_13, P_14, P_21, P_22, P_23, P_24, P_31, P_32, P_33, P_34 = symbols('P_11 P_12 P_13 P_14 P_21 P_22 P_23 P_24 P_31 P_32 P_33 P_34')
# P = Matrix([[P_11, P_12, P_13, P_14], [P_21, P_22, P_23, P_24],[ P_31, P_32, P_33, P_34]])


# Variáveis compostas
R_11 = cos(yaw)*cos(pitch)
R_12 = cos(yaw)*sin(pitch)*sin(roll) - sin(yaw)*cos(roll)
R_13 = cos(yaw)*sin(pitch)*cos(roll) + sin(yaw)*sin(roll)
R_21 = sin(yaw)*cos(pitch)
R_22 = sin(yaw)*sin(pitch)*sin(roll) + cos(yaw)*cos(roll)
R_23 = sin(yaw)*sin(pitch)*cos(roll) - cos(yaw)*sin(roll)
R_31 = -sin(pitch)
R_32 = cos(pitch)*sin(roll)
R_33 = cos(pitch)*cos(roll)
T = Matrix([[R_11, R_21, R_31], [R_12, R_22, R_32], [R_13, R_23, R_33], [t_x, t_y, t_z]]).T
K = Matrix([[f_x, 0, c_x],[0, -f_y, c_y], [0, 0, 1]])
X_vec= Matrix([X, Y, Z, 1])
u_vec = Matrix([u, v])

# Vetores calculados
P = K * T
P_vec = P.reshape(12, 1)
reproj = P*X_vec
u_pred = Matrix([reproj[0,0]/reproj[2,0], reproj[1,0]/reproj[2,0]])
zero = qui*(u_vec - u_pred)

zero

Matrix([
[χ*(u - (X*(-c_x*sin(φ) + f_x*cos(κ)*cos(φ)) + Y*(c_x*sin(ω)*cos(φ) + f_x*(-sin(κ)*cos(ω) + sin(φ)*sin(ω)*cos(κ))) + Z*(c_x*cos(φ)*cos(ω) + f_x*(sin(κ)*sin(ω) + sin(φ)*cos(κ)*cos(ω))) + c_x*t_z + f_x*t_x)/(-X*sin(φ) + Y*sin(ω)*cos(φ) + Z*cos(φ)*cos(ω) + t_z))],
[ χ*(v - (X*(-c_y*sin(φ) - f_y*sin(κ)*cos(φ)) + Y*(c_y*sin(ω)*cos(φ) - f_y*(sin(κ)*sin(φ)*sin(ω) + cos(κ)*cos(ω))) + Z*(c_y*cos(φ)*cos(ω) - f_y*(sin(κ)*sin(φ)*cos(ω) - sin(ω)*cos(κ))) + c_y*t_z - f_y*t_y)/(-X*sin(φ) + Y*sin(ω)*cos(φ) + Z*cos(φ)*cos(ω) + t_z))]])

#### <b>Ajustamento pelo método paramétrico para sistema de equações não lineares:</b>

$F(X_a) = F(X_0 + X) = L_a = L_b + V $, onde:
- $F$ é uma função não linear;
- $L_b$ é o vetor de observações com erros;
- $X_a$ é o vetor de incógnitas ajustadas;
- $L_a$ é o vetor de observações ajustadas;
- $V$ é o vetor de resíduos;
- $X_0$ é vetor com valores aproximados dos parâmetros, converge para $X_a$ após algumas iterações;
- $X$ é o Vetor correção (incremento) dentro de uma iteração, converge para $0$ após algumas iterações;

Neste caso, expandindo o termo $L_b + V = F(X_0 + X)$ pela série de Taylor e
desprezando os termos maiores ou iguais a 2, tem-se:

$$
L_b + V = F(X_0 + X) ≈ \underbrace{F(X_0)}_{L_0}+ \underbrace{\frac{dF}{dX} \bigg\rvert_{X=X_0}}_{A}  (X_a-X_0) \\
\Phi = V^TV = (\underbrace{L_0-L_b}_{L}+A\underbrace{(X_a-X_0)}_{X})^T(\underbrace{L_0-L_b}_{L}+A\underbrace{(X_a-X_0)}_{X}) = (AX+L)^T(AX+L) = min  \\
\frac{d\Phi}{dX} \bigg\rvert_{X=X_a} = 2A^TAX + 2A^TL = 0 \\
X = -(A^TA)^{-1}A^TL \ \ \ \ (1)
$$

- $A$ é a matriz das derivadas parcias da função não linear em relação às incógnitas (parâmetros);
- $L_0$ é a função dos parâmetros aproximados, converge para $L_a$ após algumas iterações;;
- $V$ é o vetor de resíduos;
- $V=AX+L$ é o modelo linearizado do método paramétrico.

O resultado da equação (1) é utilizada para atualizar o valor de $X_0$ e $L_0$ para a iteração seguinte:

$$
X_a = X^{(2)}_0 = X^{(1)} + X_0^{(1)} \\
L_a = L^{(2)}_0 = AX^{(1)} + L_0^{(1)}  \\
L^{(2)} = L^{(2)}_0 - L_b  \\
A^{(2)} = \frac{dF}{dX} \bigg\rvert_{X=X^{(2)}_0} \\
X^{(2)} = -({A^{(2)}}^TA^{(2)})^{-1}{A^{(2)}}^TL^{(2)}   \ \ \ \ (2)
$$

E assim até que $X^{(iter)}$ se reduza a 0 com determinado grau de tolerânica arbitrário.

Para encaixar nosso problema nesta modelagem, partindo da equação $F(X)$, temos:

$$
F(\Theta_a ) = F(X_a) = 
\begin{bmatrix}
  F_1(X_a) \\  
  F_2(X_a) \\ 
  \vdots \\
  F_{2NO-1}(X_a)  \\
  F_{2NO}(X_a) 
\end{bmatrix} = 
\left[
\begin{matrix} 
    \mathcal{X}_{11}\frac{P^1_{11} X_1 + P^1_{12} Y_1 + P^1_{13} Z_1 + P^1_{14}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}} \\
    \mathcal{X}_{11}\frac{P^1_{21} X_1 + P^1_{22} Y_1 + P^1_{23} Z_1 + P^1_{24}}{P^1_{31} X_1 + P^1_{32} Y_1 + P^1_{33} Z_1 + P^1_{34}}  \\
    \vdots \\
    \mathcal{X}_{NO} \frac{P^N_{11} X_O + P^N_{12} Y_O + P^N_{13} Z_O + P^N_{14}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}}   \\
    \mathcal{X}_{NO} \frac{P^N_{21} X_O + P^N_{22} Y_O + P^N_{23} Z_O + P^N_{24}}{P^N_{31} X_O + P^N_{32} Y_O + P^N_{33} Z_O + P^N_{34}} 
\end{matrix}
\right]  \\
L_b+V = 
\begin{bmatrix} 
    \mathcal{X_{11}}u_{11}  + e_{1} \\ 
    \mathcal{X}_{11}v_{11}  + e_{2}\\
    \vdots   \\
    \mathcal{X}_{NO}u_{NO} + e_{2NO-1}\\ 
    \mathcal{X}_{NO}v_{NO}  + e_{2NO}
\end{bmatrix}
$$

Assim:

$$
A =
\begin{bmatrix}
\frac{dF_1}{dX_a} \\
\frac{dF_2}{dX_a} \\
\vdots \\
\frac{dF_{2NO-1}}{dX_a} \\
\frac{dF_{2NO}}{dX_a} \\
\end{bmatrix} =
\begin{bmatrix}
  \frac{\partial F_1}{\partial f_x} & \frac{\partial F_1}{\partial f_y} & \dots & \frac{\partial F_1}{\partial X_M} & \frac{\partial F_1}{\partial Y_M} &\frac{\partial F_1}{\partial Z_M} \\
  \frac{\partial F_2}{\partial f_x} & \frac{\partial F_2}{\partial f_y} & \dots & \frac{\partial F_2}{\partial X_M} & \frac{\partial F_2}{\partial Y_M} &\frac{\partial F_2}{\partial Z_M} \\
  \vdots & \vdots  & \ddots & \vdots & \vdots & \vdots \\
  \frac{\partial F_{2NO-1}}{\partial f_x} & \frac{\partial F_{2NO-1}}{\partial f_y} & \dots & \frac{\partial F_{2NO-1}}{\partial X_M} & \frac{\partial F_{2NO-1}}{\partial Y_M} &\frac{\partial F_{2NO-1}}{\partial Z_M} \\
  \frac{\partial F_{2NO}}{\partial f_x} & \frac{\partial F_{2NO}}{\partial f_y} & \dots & \frac{\partial F_{2NO}}{\partial X_M} & \frac{\partial F_{2NO}}{\partial Y_M} &\frac{\partial F_{2NO}}{\partial Z_M} \\
\end{bmatrix}_{2NO \times (4 + 6N + 3M)}
$$

Arbitrando $X_0^{(iter = 1)} = [0] $, obtemos $L^{(1)}_0 = F(X^{(1)}_0)$ para a 1ª iteração, devemos encontrar o vetor de correção $X^{(1)}$ através do método descrito pela equação (2), repetindo os cálculos até a tolerânica desejada.

### Exemplo 2 - Coelho (2007)

Considere o bloco das 6 imagens e 16 Pontos 3D abaixo relacionados:

<img src="data/bloco.png" width=600>

No caso da fig. acima, temos pontos 3D fotogramétricos $X_j$ para $j=5,...,16$ e pontos 3D de controle $X_k$ para $k=1,...,4$.

$\mathbf{X_1}, \mathbf{X_5}, \mathbf{X_6}, \mathbf{X_8}, \mathbf{X_9}, \mathbf{X_{10}}$ podem ser vistos na imagem 1, portanto devemos adquirir suas coordenadas do espaço objeto, ou seja, $\mathbf{u_{11}}, \mathbf{u_{15}}, \mathbf{u_{16}}, \mathbf{u_{18}}, \mathbf{u_{19}},\mathbf{u_{1-10}}$, bem como organizar o relê $\mathcal{X}$ da seguinte maneira $[\mathcal{X_{11}}, \mathcal{X_{15}}, \mathcal{X_{16}}, \mathcal{X_{18}}, \mathcal{X_{19}},\mathcal{X_{1-10}}] =1$ e nos demais casos, 0.

$\mathbf{X_1}, \mathbf{X_3}, \mathbf{X_5}, \mathbf{X_6}, \mathbf{X_8}, \mathbf{X_9},\mathbf{X_{10}},  \mathbf{X_{13}}, \mathbf{X_{14}}, \mathbf{X_{15}}$ podem ser vistos na imagem 2, portanto devemos adquirir suas coordenadas do espaço objeto, ou seja, 
$\mathbf{u_{21}}, \mathbf{u_{25}}, \mathbf{u_{26}},\mathbf{u_{28}}, \mathbf{u_{29}}, \mathbf{u_{2-10}}, \mathbf{u_{2-13}}, \mathbf{u_{2-14}}, \mathbf{u_{2-15}} $, bem como organizar o relê $\mathcal{X}$ da seguinte maneira 
$[\mathcal{X_{21}}, \mathcal{X_{25}}, \mathcal{X_{26}}, \mathcal{X_{28}}, \mathcal{X_{29}}, \mathcal{X_{2-10}}, \mathcal{X_{2-13}}, \mathcal{X_{2-14}}, \mathcal{X_{2-15}}] =1$ e nos demais casos, 0. E assim por diante.

A matriz das derivadas parciais $A$ seria do formato $(2NO \times 4+6N+3M) = (2*6*16 \times 4+6*6+3*4)  = ( 192 \times 52)$ .

## 2. Técnicas de coorrespondência automática de pontos entre imagens sobrepostas

Existem diversos algoritmos ques objetivam buscar por pontos correspondentes: 
- Scale Invariant Feature Transform (SIFT);
- Speeded-Up Robust Features (SURF);
- Oriented FAST and Rotated BRIEF (ORB);
- Detector-Free Local Feature Matching with Transformers (LoFTR) - https://github.com/zju3dv/LoFTR;
- Self-Supervised Interest Point Detection and Description (SuperPoint) - https://github.com/rpautrat/SuperPoint;
- Grid-Based Motion Statistics for Fast, Ultra-Robust Feature Correspondence (GMS) - https://github.com/germain-hug/GMS-Feature-Correspondence
- ...

<img src="data/SIFT.png">

O ganho em quantidade de pontos correspondentes é seguido pela inserção de outliers indesejados.

### Exemplo

<b>SIFT</b>

In [ ]:
# Load the images
img = []
img.append(f'aux_data/FOTOS_VOO_2GPTE/DJI_0019.JPG')
img.append(f'aux_data/FOTOS_VOO_2GPTE/DJI_0020.JPG')

import matplotlib.pyplot as plt
import numpy as np
import cv2


###### Resize to be more fast
def resize(img):
    original_height, original_width = img.shape[:2]
    
    # Define new width while maintaining the aspect ratio
    new_width = 600
    aspect_ratio = new_width / original_width
    new_height = int(original_height * aspect_ratio)  # Compute height based on aspect ratio
    
    # Resize the image
    return cv2.resize(img, (new_width, new_height))


img0_raw = cv2.imread(img[0])
img0_raw = resize(img0_raw)
img0_rgb = cv2.cvtColor(img0_raw, cv2.COLOR_BGR2RGB)
img0_gray = cv2.cvtColor(img0_raw, cv2.COLOR_BGR2GRAY)

img1_raw = cv2.imread(img[1])
img1_raw = resize(img1_raw)
img1_rgb = cv2.cvtColor(img1_raw, cv2.COLOR_BGR2RGB)
img1_gray = cv2.cvtColor(img1_raw, cv2.COLOR_BGR2GRAY)

# Detect keypoints and compute descriptors
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
kp0, des0 = sift.detectAndCompute(img0_gray,None)
img0 = cv2.drawKeypoints(img0_rgb, kp0, img0_raw)
kp1, des1 = sift.detectAndCompute(img1_gray,None)
img1 = cv2.drawKeypoints(img1_rgb, kp1, img1_raw)

bf = cv2.BFMatcher()
# Match descriptors.
matches = bf.match(des0,des1)
# Sort them in the order of their distances. --> Lower is better
matches = sorted(matches, key = lambda x:x.distance)
print("Matches",len(matches))
k = 10
print(f"Mostrabdo os {k} melhores:")

img3 = cv2.drawMatches(img0_rgb,kp0,img1_rgb,kp1,matches[:k],None,
                              matchColor=(0, 0, 0),
                              flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

f, a = plt.subplots(figsize = (20,15))
# Draw a vertical line at the half of the width
a.axvline(x=img0_raw.shape[1], color='red', linestyle='--', linewidth=2)
# a.axis('off')
plt.imshow(img3)
plt.show()

list_kp0 = [kp0[mat.queryIdx].pt for mat in matches]
list_kp1 = [kp1[mat.trainIdx].pt for mat in matches]

for i in range(0, len(matches[:k])):
    print(list_kp0[i],"=",list_kp1[i])